# model の重要度みる

In [10]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%autoreload 2

import os
import sys
import json
import pickle

from numerapi import NumerAPI
from numerai_tools.scoring import numerai_corr, correlation_contribution

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

from scipy.stats import kurtosis, skew

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import IncrementalPCA

sys.path.append("../")

from src.eda import plot_feature_dist
from src.model import MLModel, LightGBMModel
from src.validation import caluculate_metrics
from src.featuring import feature_pca

In [12]:
class cfg:
    dataset_path = "../dataset/v5.0"
    feature_set = "medium"
    threshold_corr = 0.9
    exp = "eda_20250212"
    target_exp = "exp015"

os.makedirs(cfg.exp, exist_ok=True)


In [13]:
model = LightGBMModel()
model.load_model(os.path.join(cfg.target_exp, "model_0_target.pkl"))


In [14]:
df = pd.DataFrame({
    "feature": model.model.feature_name(),
    "score": model.model.feature_importance()
}).sort_values("score", ascending=False)


In [15]:
for i in range(len(df) // 5):
    
    display(df.head((i+1)*5).tail(5))


,feature,score
400,per_era_count,384
42,feature_australian_unstigmatized_pasch,253
74,feature_cephalate_unfine_molality,237
56,feature_boggy_parthia_styrene,236
65,feature_burned_sortable_vicar,227


,feature,score
4,feature_acetose_crackerjack_needlecraft,220
44,feature_awash_reverberative_isatin,217
61,feature_brummagem_freckliest_papaverine,214
55,feature_blurred_divertible_desensitizer,207
70,feature_canine_umpteenth_bidding,196


,feature,score
37,feature_asyndetic_unwieldy_glossolalia,195
66,feature_busy_clear_divi,195
16,feature_alarming_forenamed_shearing,194
68,feature_cabbalistic_unadmitted_ouagadougou,193
77,feature_chaste_disqualifying_intermodulation,189


,feature,score
46,feature_axile_complexioned_tartarization,187
60,feature_bronzy_thermoluminescent_extraversion,184
73,feature_celibate_wayworn_cosmism,184
53,feature_binding_mexican_sunk,183
1,feature_ablest_inflexional_egeria,182


,feature,score
39,feature_atmospherical_undisciplinable_enterostomy,182
59,feature_brittle_madagascar_peak,174
3,feature_accoutered_revolute_vexillology,172
58,feature_brickier_photosensitive_bodement,172
45,feature_axial_solutrean_pry,171


...

,feature,score
149,per_era_max_feature_aliunde_unhaunted_coacervate,0
146,per_era_min_feature_alarming_forenamed_shearing,0
145,per_era_max_feature_alarming_forenamed_shearing,0
316,per_era_mean_feature_brittle_madagascar_peak,0
317,per_era_max_feature_brittle_madagascar_peak,0


,feature,score
158,per_era_min_feature_analgesic_pensionary_exterior,0
294,per_era_min_feature_binding_mexican_sunk,0
269,per_era_max_feature_baculine_panoplied_aerobio...,0
280,per_era_mean_feature_bavarian_educible_fetishism,0
270,per_era_min_feature_baculine_panoplied_aerobio...,0


,feature,score
177,per_era_max_feature_antidepressant_rationalist...,0
174,per_era_min_feature_anthracoid_tamable_consulship,0
173,per_era_max_feature_anthracoid_tamable_consulship,0
276,per_era_mean_feature_barkier_lackluster_hunker,0
277,per_era_max_feature_barkier_lackluster_hunker,0


,feature,score
278,per_era_min_feature_barkier_lackluster_hunker,0
281,per_era_max_feature_bavarian_educible_fetishism,0
293,per_era_max_feature_binding_mexican_sunk,0
282,per_era_min_feature_bavarian_educible_fetishism,0
170,per_era_min_feature_antediluvian_establishment...,0


,feature,score
169,per_era_max_feature_antediluvian_establishment...,0
162,per_era_min_feature_analogical_obstructed_mart...,0
289,per_era_max_feature_beholden_ithyphallic_ankle,0
290,per_era_min_feature_beholden_ithyphallic_ankle,0
161,per_era_max_feature_analogical_obstructed_mart...,0
